In [11]:
!pip3 install jsonlines
!pip3 install ipython-autotime

%load_ext autotime

!pip3 install sentence_transformers

import pandas as pd
import numpy as np
import re
from nltk.stem import WordNetLemmatizer
import nltk
from tqdm.notebook import tqdm
tqdm.pandas()
import requests

from termcolor import colored
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

my_dir='/YOUR-DIR/'


import os
from datetime import datetime
import pickle
import ast
import re

import tensorflow as tf
import tensorflow_hub as hub
from sentence_transformers import SentenceTransformer

from typing import List
import jsonlines as jl
import webbrowser

lemmatizer = WordNetLemmatizer()
# punctuations = """!"$%()*+,*/:#»«'";“<=>?[\]^`”{.}~"""
punctuations = """!"$%()*+,*/:»«";“<=>?[\]^`”|{.}~"""
def clean_text_local(row):
    row = str(row)
    row=re.sub('[^\x00-\x7F]+',' ',row)
    row=re.sub('\x0c',' ',row)
    row=re.sub('\u200c',' ',row)
    # row=re.sub('\n\n',' . ',row)
    # row=re.sub('\\n',' ',row)

    row=re.sub('\\\\n',' ',row)
    row=re.sub('\n',' ',row)

    row=re.sub("-?NEWLINE_TOKEN", " ",row)
    row=re.sub("TAB_TOKEN", " ",row)
    row=re.sub("Alternate option=", "",row)
    row=re.sub('^RT',' ',row)
    row = row.lower()
    row=re.sub("@[A-Za-z0-9_]+","",row)
    row=re.sub("http\S+|www.\S+","",row)

    row = re.sub(r'<.*?>', '', row)
    row=re.sub("&amp","&",row)
#     row=re.sub("\d","",row)
#     row=re.sub(r"\b\d+\b|\b(?![i])[a-z]\b","",row) # removes every isolated number or char except i
    # row = ' '.join([lemmatizer.lemmatize(w) for w in nltk.word_tokenize(row)])
    row=re.sub('-',' ',row)
    no_punct = ""
    for char in row:
        if char not in punctuations:
            no_punct = no_punct + char
    no_punct = no_punct.strip()
    no_punct = re.sub('\s+',' ',no_punct)
    return no_punct
    # return row

model_st1 = SentenceTransformer('all-mpnet-base-v2')
print('loaded all-mpnet-base-v2', flush=True)


def embed(model, model_type, sentences):
    """
    wrapper function for generating message embeddings
    """
    if model_type == 'use':
        embeddings = model(list(sentences))
    elif model_type == 'sentence transformer':
        embeddings = model.encode(list(sentences),show_progress_bar=False)

    return embeddings

def split_embed(model_, model_type_, all_sentences_, n_partitions_=20):
    all_sentences_ = np.array_split(all_sentences_, indices_or_sections = n_partitions_)
    for i, new_sentences in enumerate(all_sentences_):
        if i==0:
            # print(colored('\nbeginning section {}'.format(i+1),'yellow'),flush=True)
            all_embeddings = embed(model_, model_type_, new_sentences)
        if i>0:
            # print(colored('\nbeginning section {}'.format(i+1),'yellow'),flush=True)
            new_embeddings = embed(model_, model_type_, new_sentences)
            all_embeddings = np.concatenate((all_embeddings,new_embeddings),axis=0)
    return all_embeddings

def vector_to_column(df,vectors,column_name):
    df[column_name]=0
    df[column_name]=df[column_name].astype(object)
    df.reset_index(drop=True,inplace=True)
    indices=df.index
    for i in indices:
        df.at[i, column_name]=vectors[i]
    return df


def create_timeline(in_file_path):
    # List the name of the text files in the "in_file_path"
    lst = os.listdir(in_file_path)#[0:500]
    user_dict = {}
    for file in lst:
        with open(in_file_path+"/"+str(file)) as fr:
            concat_tweets = {}
            # Read the tweets objects line by line
            line = fr.readline()
            tweets = []
            while(line):
                # Change it to dict
                line = ast.literal_eval(line)
                screen_name = line['user']['screen_name']
                user_id = line['user']['id']
                tweets.append(line)
                line = fr.readline()
            concat_tweets = {"screen_name" : screen_name,
                            "user_id" : user_id,
                            "recent_tweets" : tweets}
        user_timeline = []
        twt_time = []
        retweeted = []
        screen_name = []
        user_dict[str(concat_tweets['user_id'])] = {}
        # Add all the users and their requested info (here it is tweets, the time, and whether it is a retweet) to the "concat_tweets" dictionary one by one.
        # You can add more features by looking into the tweet object, if needed.
        if concat_tweets['recent_tweets'] is not None:
            for tweet in concat_tweets['recent_tweets']:
                user_timeline.append(tweet['full_text'])
                twt_time.append(datetime.strptime(tweet['created_at'], "%a %b %d %H:%M:%S +0000 %Y"))
                retweeted.append(tweet['retweeted'])
        user_dict[str(concat_tweets['user_id'])]['tweets'] = user_timeline
        user_dict[str(concat_tweets['user_id'])]['twt_time'] = twt_time
        user_dict[str(concat_tweets['user_id'])]['retweeted'] = retweeted
    return user_dict

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
loaded all-mpnet-base-v2
time: 23 s (started: 2023-11-04 00:32:38 +00:00)


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
time: 15.3 s (started: 2023-11-04 00:27:40 +00:00)


In [3]:
# user_list_df = pd.read_hdf(f'{my_dir}/Datasets/User_lists/hashtag_users_list.hdf')
user_list_df = pd.read_hdf(f'{my_dir}/Datasets/User_lists/network_users_list.hdf')

time: 1.83 s (started: 2023-11-04 00:27:56 +00:00)


In [4]:
def get_average_embedding_from_df(data,text_column,min_sample=100):
  if len(data) == 0:
    return [0]
  # data=data[data['lang']=='en']
  # if len(data) == 0:
  #   return [0]

  # data=data[data[text_column].str.contains("RT")==False]
  # if len(data) == 0:
  #   return [0]
  # if len(data) >= 200:
  #   data=data.sample(n=200,random_state=1)
  # data = data[(data[text_column].str.contains("RT")==False) | (data[text_column].str.contains("ndamendment|gun|abortion|ukraine|russia|lifewins|mybodymychoice|roe|vade|sxsw")==False)]

  data[text_column] = data.apply(lambda x: clean_text_local(x[text_column]),axis=1)
  # data = data[(data[text_column].str.contains("rt ")==False) | (data[text_column].str.contains("ndamendment|gun|abortion|ukraine|russia|lifewins|mybodymychoice|roe|vade|sxsw")==False)]
  if topic == 'abortion':
    data=data[data[text_column].str.contains("abortion|life|choice|mybodymychoice|roe|vade|reproductive")==False]
  if topic == 'gun':
    data=data[data[text_column].str.contains("amendment|gun|shooting")==False]
  if topic == 'ukraine':
    data=data[data[text_column].str.contains("ukraine|russia")==False]
  if topic == 'sxsw':
    data=data[data[text_column].str.contains("sxsw")==False]

  # data=data[data[text_column].str.contains("ndamendment|gun|abortion|ukraine|russia|lifewins|choice|mybodymychoice|roe|vade|sxsw")==False]

  data.drop_duplicates(subset=text_column,inplace=True)
  # print('length after dropping duplicates = {}'.format(len(data)),flush=True)
  # trpCom1=trpCom1.sample(n=200000, random_state=1,replace=False)
  if len(data) == 0:
    return [0]
  data = data[data[text_column].map(len)>30]#.sample(n=100000, random_state=1,replace=False)
  data.reset_index(drop=True,inplace=True)
  print('length after removing undesirable texts = {}'.format(len(data)),flush=True)
  if len(data) >= min_sample:
    # generate embeddings for each model

    embeddings_st1 = np.asarray(split_embed(model_st1, 'sentence transformer', data[text_column], n_partitions_=1))

    return np.mean(embeddings_st1,axis=0)
  else:
    return [0]

time: 971 µs (started: 2023-11-04 00:28:06 +00:00)


In [5]:
data_path = 'Datasets/User_Tweets/'

def read_data_user_from_userid(user_id_x,my_dir=my_dir,data_path=data_path):
  print('working on user {}'.format(user_id_x))
  try:
    return pd.read_json(my_dir + data_path + str(int(user_id_x)) + '.jsonl', lines=True)
  except:
    print('File for user {} not found'.format(user_id_x))
    return pd.DataFrame()

def read_data_user_from_username_csv(username_x,my_dir=my_dir,data_path=data_path):
  print('working on user {}'.format(username_x))
  try:
    return pd.read_csv(my_dir + data_path + username_x + '.csv')
  except:
    print('File for user {} not found'.format(username_x))
    return pd.DataFrame()

time: 1.03 ms (started: 2023-11-04 00:28:13 +00:00)


In [ ]:
user_list_df['average_embedding']=user_list_df.progress_apply(lambda x: get_average_embedding_from_df(data=read_data_user_from_userid(x['user_id']),text_column='full_text',min_sample=30),axis=1)

In [ ]:
pickle.HIGHEST_PROTOCOL = 4

user_list_df.to_hdf(f'{my_dir}/Datasets/congresspeople_list.hdf','df')
user_list_df.to_hdf(f'{my_dir}/Datasets/User_lists/hashtag_users_list.hdf','df')
user_list_df.to_hdf(f'{my_dir}/Datasets/User_lists/network_users_list.hdf','df')

In [ ]:
user_list_df

,user_screen_name_old,hashtag,user_id,user_info,average_embedding
0,1215jorge,Pro-Gun,191707978,"{'id': 191707978, 'id_str': '191707978', 'name...","[-0.0046667918, 0.05530896, 0.0020495367, 0.00..."
1,125LolaLola,Pro-Gun,37808197,"{'id': 37808197, 'id_str': '37808197', 'name':...","[-0.01883486, 0.07484672, 0.0042844554, 0.0088..."
2,1Egetfl,Pro-Gun,833080577582624768,"{'id': 833080577582624768, 'id_str': '83308057...","[-0.009108344, 0.063201874, 0.0027614383, 0.00..."
3,45LVNancy,Pro-Gun,1477071867219304449,"{'id': 1477071867219304449, 'id_str': '1477071...","[-0.017589265, 0.07834912, 0.0033631946, 0.005..."
4,1libertyboi,Pro-Gun,1150064718758981632,"{'id': 1150064718758981632, 'id_str': '1150064...","[-0.014149628, 0.07611318, 0.0047517954, 0.002..."
...,...,...,...,...,...
21390,edwards_lill,StandWithUkraine,1134106780366180352,"{'id': 1134106780366180352, 'id_str': '1134106...","[-0.019277696, 0.07905936, 0.005222643, 0.0037..."
21391,MrsNDpendent,StandWithUkraine,22256226,"{'id': 22256226, 'id_str': '22256226', 'name':...","[-0.005334577, 0.07084483, 0.005182821, -0.001..."
21392,plzsirnomorety,StandWithUkraine,user not found,{},[0]
21393,sme1965,StandWithUkraine,793818497713213440,"{'id': 793818497713213440, 'id_str': '79381849...","[0.0077153877, 0.039057262, -0.010893657, 0.01..."


time: 71.1 ms (started: 2022-07-01 13:54:39 +00:00)
